In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive
!pwd
!git clone https://github.com/seujung/KoBART-summarization.git

/content/drive/MyDrive
/content/drive/MyDrive
Cloning into 'KoBART-summarization'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 142 (delta 46), reused 37 (delta 37), pack-reused 83
Receiving objects: 100% (142/142), 37.23 MiB | 16.06 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [ ]:
%cd /content/drive/MyDrive/KoBART-summarization
!pip install pytorch-lightning

/content/drive/MyDrive/KoBART-summarization
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 800 kB 18.6 MB/s 
     |████████████████████████████████| 125 kB 63.5 MB/s 
     |████████████████████████████████| 512 kB 65.9 MB/s 


In [ ]:
!python train.py --gradient_clip_val 1.0  \
                 --max_epochs 50 \
                 --default_root_dir logs \
                 --gpus 1 \
                 --batch_size 8 \
                 --num_workers 4

In [ ]:
import argparse
import logging
import os
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
from pytorch_lightning import loggers as pl_loggers
from torch.utils.data import DataLoader, Dataset
from dataset import KobartSummaryModule
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup


class ArgsBase():
    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = argparse.ArgumentParser(
            parents=[parent_parser], add_help=False)
        parser.add_argument('--train_file',
                            type=str,
                            default='data/train.tsv',
                            help='train file')

        parser.add_argument('--test_file',
                            type=str,
                            default='data/test.tsv',
                            help='test file')

        parser.add_argument('--batch_size',
                            type=int,
                            default=14,
                            help='')
        parser.add_argument('--max_len',
                            type=int,
                            default=512,
                            help='max seq len')
        return parser

class Base(pl.LightningModule):
    def __init__(self, hparams, trainer, **kwargs) -> None:
        super(Base, self).__init__()
        # self.save_hyperparameters(hparams)
        self.trainer = trainer

    @staticmethod
    def add_model_specific_args(parent_parser):
        # add model specific args
        parser = argparse.ArgumentParser(
            parents=[parent_parser], add_help=False)

        parser.add_argument('--batch-size',
                            type=int,
                            default=14,
                            help='batch size for training (default: 96)')

        parser.add_argument('--lr',
                            type=float,
                            default=3e-5,
                            help='The initial learning rate')

        parser.add_argument('--warmup_ratio',
                            type=float,
                            default=0.1,
                            help='warmup ratio')

        parser.add_argument('--model_path',
                            type=str,
                            default=None,
                            help='kobart model path')
        return parser
    
    def setup_steps(self, stage=None):
        # NOTE There is a problem that len(train_loader) does not work.
        # After updating to 1.5.2, NotImplementedError: `train_dataloader` · Discussion #10652 · PyTorchLightning/pytorch-lightning https://github.com/PyTorchLightning/pytorch-lightning/discussions/10652
        train_loader = self.trainer._data_connector._train_dataloader_source.dataloader()
        return len(train_loader)

    def configure_optimizers(self):
        # Prepare optimizer
        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(
                nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in param_optimizer if any(
                nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        optimizer = AdamW(optimizer_grouped_parameters,
                          lr=self.hparams.lr, correct_bias=False)
        num_workers = self.hparams.num_workers

        data_len = self.setup_steps(self)
        logging.info(f'number of workers {num_workers}, data length {data_len}')
        num_train_steps = int(data_len / (self.hparams.batch_size * num_workers) * self.hparams.max_epochs)
        logging.info(f'num_train_steps : {num_train_steps}')
        num_warmup_steps = int(num_train_steps * self.hparams.warmup_ratio)
        logging.info(f'num_warmup_steps : {num_warmup_steps}')
        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)
        lr_scheduler = {'scheduler': scheduler, 
                        'monitor': 'loss', 'interval': 'step',
                        'frequency': 1}
        return [optimizer], [lr_scheduler]


class KoBARTConditionalGeneration(Base):
    def __init__(self, hparams, trainer=None, **kwargs):
        super(KoBARTConditionalGeneration, self).__init__(hparams, trainer, **kwargs)
        self.model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v1')
        self.model.train()
        self.bos_token = '<s>'
        self.eos_token = '</s>'
        
        self.tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')
        self.pad_token_id = self.tokenizer.pad_token_id

    def forward(self, inputs):

        attention_mask = inputs['input_ids'].ne(self.pad_token_id).float()
        decoder_attention_mask = inputs['decoder_input_ids'].ne(self.pad_token_id).float()
        
        return self.model(input_ids=inputs['input_ids'],
                          attention_mask=attention_mask,
                          decoder_input_ids=inputs['decoder_input_ids'],
                          decoder_attention_mask=decoder_attention_mask,
                          labels=inputs['labels'], return_dict=True)


    def training_step(self, batch, batch_idx):
        outs = self(batch)
        loss = outs.loss
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        outs = self(batch)
        loss = outs['loss']
        return (loss)

    def validation_epoch_end(self, outputs):
        losses = []
        for loss in outputs:
            losses.append(loss)
        self.log('val_loss', torch.stack(losses).mean(), prog_bar=True)



In [ ]:
# args = parser.parse_args()

import easydict

args = easydict.EasyDict({
        "mode" : 'train',
        "model_file" : "model/scae.model",
        "batch_size" : 128,
        "epochs" : 20,
        "lr" : 1e-5,
        "K" :24,
        "C" : 10,
        "load_ext": "store_true"
    })
parser = argparse.ArgumentParser(description='KoBART Summarization')

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')


dm = KobartSummaryModule('data/train.tsv',
                    'data/test.tsv',
                    tokenizer,
                    batch_size=14,
                    max_len=512,
                    num_workers=4)

checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor='val_loss',
                                                    dirpath='logs',
                                                    filename='model_chp/{epoch:02d}-{val_loss:.3f}',
                                                    verbose=True,
                                                    save_last=True,
                                                    mode='min',
                                                    save_top_k=3)

tb_logger = pl_loggers.TensorBoardLogger(os.path.join('logs', 'tb_logs'))
lr_logger = pl.callbacks.LearningRateMonitor()
trainer = pl.Trainer.from_argparse_args(args, logger=tb_logger,
                                        callbacks=[checkpoint_callback, lr_logger])

model = KoBARTConditionalGeneration(args, trainer)
model

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


KoBARTConditionalGeneration(
  (model): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(30000, 768, padding_idx=3)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(30000, 768, padding_idx=3)
        (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
        (layers): ModuleList(
          (0): BartEncoderLayer(
            (self_attn): BartAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=

In [ ]:
!pip install -U --no-cache-dir gdown --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
!gdown 1-su5yF8EMOgwEvUq-tiFBA92dEFhwIW6

Downloading...
From: https://drive.google.com/uc?id=1-su5yF8EMOgwEvUq-tiFBA92dEFhwIW6
To: /content/drive/MyDrive/KoBART-summarization/epoch=00-val_loss=1.616.ckpt
100% 1.49G/1.49G [00:07<00:00, 212MB/s]


In [ ]:
model2 = model.load_from_checkpoint('/content/drive/MyDrive/KoBART-summarization/epoch=00-val_loss=1.616.ckpt')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [38]:
import json
import os
import glob
import pandas as pd
from tqdm.notebook import tqdm
import zipfile
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from IPython import display

In [37]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset

ds = load_dataset("com0040/only")
ds

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['original', 'summary'],
        num_rows: 12150
    })
})

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.tsv", delimiter='\t')
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test.tsv", delimiter='\t')

new_tf_dataset = tf.data.Dataset.from_tensor_slices((df_train['news'].values, df_train['summary'].values))

In [ ]:
new_tf_dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("psyche/KoT5-summarization")
tokenizer

Downloading:   0%|          | 0.00/2.52k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.88M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

PreTrainedTokenizerFast(name_or_path='psyche/KoT5-summarization', vocab_size=32100, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id

In [ ]:
df_train

,news,summary
0,8일 인천 영종도 스카이72 골프앤리조트 오션 코스. 한국 여자프로골프(KLPG...,안 선수가 한국 여자프로골프 투어 하나금융그룹 챔피언십에서 우승했다.
1,바이든 미국 대통령 당선 - 한국에 주는 교훈미국 대선에서 드러난 지지층의 ‘영끌’...,미국 대선에서 극단 정치인 치킨게임이 벌어진 데에 대한 책임은 도널드 트럼프 대통령...
2,"처음 한국은 상·하반기에 한 번씩 1년에 두 차례 정도를 생각했는데, 블링컨이 먼...",아랍의 봄 사태 때 핵심적 역할을 했던 라이스 전 백악관 국가안보보좌관은 행정부의 ...
3,"이제 44세지만, 그를 만나본 이들은 하나같이 “진정한 브레인”이라고 입을 모은다...",트럼프 행정부의 국방장관직을 고사했던 웨스트이그젝 어드바이저스의 공동 설립자인 플러...
4,한 학기의 반환점 격인 중간시험이 끝나면 좋아하는 시를 소개하는 시간을 가진다. ...,국부론을 쓴 스미스는 인간의 이기심과 경쟁심에도 불구하고 공감의 원리가 인간의 심성...
...,...,...
21595,설날 가스폭발로 일가족 5명이 숨지는 등 9명의 사상자를 낸 강원 동해시 펜션 사...,가스 폭발 사고가 난 묵호진동 펜션은 펜션 영업 신고를 하지 않은 불법 숙박업소였다.
21596,"연휴 마지막날인 27일, ‘미니 태풍’ 수준의 강풍에 제주도와 남해상에 강풍‧풍랑...","제주도와 남해상에 강풍·풍랑주의보가 내려진 가운데, 제주도는 많은 비가 내리겠고 경..."
21597,김정은 북한 국무위원장이 고모이자 처형된 장성택의 부인 김경희 전 노동당 비서를 6...,태 전 공사는 블로그를 통해 김 북한 국무위원장이 김 전 비서를 6년 만에 등장시킨...
21598,20년 전 세계 최초로 액정(LCD) TV를 선보였던 일본 샤프가 LG디스플레이로부...,도쿄올림픽을 앞두고 일본에서 화질이 좋은 TV 수요가 증가함에 따라 일본 샤프가 L...


In [ ]:
import re
 
# reading given tsv file
with open("/content/drive/MyDrive/Colab Notebooks/test.tsv", 'r') as myfile: 
  with open("/content/drive/MyDrive/Colab Notebooks/test_news.csv", 'w') as csv_file:
    for line in myfile:
       
      # Replace every tab with comma
      fileContent = re.sub("\t", ",", line)
       
      # Writing into csv file
      csv_file.write(fileContent)
 
# output
print("Successfully made csv file")

Successfully made csv file


In [ ]:
read_csv_df = pd.read_table("/content/drive/MyDrive/Colab Notebooks/test.tsv", sep='\t')
read_csv_df.to_csv('/content/drive/MyDrive/Colab Notebooks/test.csv', index=False)

In [3]:
import pandas as pd
read_csv_df = pd.read_table("/content/drive/MyDrive/Alpaco_NLP/train.tsv", sep='\t')
read_csv_df.to_csv('/content/drive/MyDrive/Alpaco_NLP/train.csv', index=False)

In [39]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/Alpaco_NLP/train.csv")

  0%|          | 0/1 [00:00<?, ?it/s]

In [40]:
max_input_length = 1024
max_target_length = 128


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["news"], max_length=max_input_length, truncation=True
    )
    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/22 [00:00<?, ?ba/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['news', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 21600
    })
})

In [41]:
# test size will be 15% of train dataset
test_size=.15

processed_dataset = tokenized_datasets['train'].shuffle().train_test_split(test_size=test_size)
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['news', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 18360
    })
    test: Dataset({
        features: ['news', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3240
    })
})

In [10]:
%cd /content/drive/MyDrive/Alpaco_NLP

/content/drive/MyDrive/Alpaco_NLP


In [17]:
!pip install -U --no-cache-dir gdown --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [18]:
!gdown 1-VCfwW353NVoHkQN1wVKvca98PcyedRd

Downloading...
From: https://drive.google.com/uc?id=1-VCfwW353NVoHkQN1wVKvca98PcyedRd
To: /content/drive/MyDrive/Alpaco_NLP/tf_model.h5
100% 1.09G/1.09G [00:06<00:00, 168MB/s]


In [42]:
from transformers import TFAutoModelForSeq2SeqLM
# load pre-trained model
model = TFAutoModelForSeq2SeqLM.from_pretrained('psyche/KoT5-summarization',from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight', 'lm_head.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [19]:
from transformers import DataCollatorForSeq2Seq

train_batch_size = 1
eval_batch_size = 1

# Data collator that will dynamically pad the inputs received, as well as the labels.
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

# converting our train dataset to tf.data.Dataset
tf_train_dataset = processed_dataset["train"].to_tf_dataset(
   columns=["input_ids", "attention_mask", "labels"],
   shuffle=True,
   batch_size=train_batch_size,
   collate_fn=data_collator)

# converting our test dataset to tf.data.Dataset
tf_eval_dataset = processed_dataset["test"].to_tf_dataset(
   columns=["input_ids", "attention_mask", "labels"],
   shuffle=True,
   batch_size=eval_batch_size,
   collate_fn=data_collator)


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [48]:
import tensorflow as tf 
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

# compile model
model.compile(optimizer=adam)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [22]:
import os
# 체크포인트 파일 경로 설정
checkpoint_path = "/content/drive/MyDrive/Alpaco_NLP/ckpt/" 
checkpoint_dir = os.path.dirname(checkpoint_path) # 현재 경로 폴더 반환
checkpoint_dir

'/content/drive/MyDrive/Alpaco_NLP/ckpt'

In [46]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only= True,
                                                 monitor='val_loss',
                                                 save_best_only=True,
                                                 )

In [ ]:
import evaluate
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

def compute_metrics(pred):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'auroc': auc,
        'rouge': metric.compute(predictions=predictions, references=labels)
    }

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_eval_dataset)

In [47]:
callbacks = [cp_callback]

In [49]:
train_results = model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    epochs=5,
    callbacks = callbacks,
)


Epoch 1/5
18360/18360 [==============================] - 2094s 112ms/step - loss: 1.2184 - val_loss: 1.0923
Epoch 2/5
18360/18360 [==============================] - 2065s 112ms/step - loss: 1.0520 - val_loss: 1.1257
Epoch 3/5
18360/18360 [==============================] - 2088s 114ms/step - loss: 0.9144 - val_loss: 1.1701
Epoch 4/5
18360/18360 [==============================] - 2086s 114ms/step - loss: 0.7967 - val_loss: 1.2134
Epoch 5/5
18360/18360 [==============================] - 2078s 113ms/step - loss: 0.7026 - val_loss: 1.2702


In [51]:
model.save_weights('/content/drive/MyDrive/Alpaco_NLP/ckpt2')

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/ckpt/save'
model.save_pretrained(path)
# model = TFPreTrainedModel.from_pretrained(path)

In [ ]:
from transformers import TFPreTrainedModel

model = TFPreTrainedModel.from_pretrained('/content/drive/MyDrive/Colab Notebooks/ckpt/save')

AttributeError: ignored

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=5,   # batch size per device during training
    per_device_eval_batch_size=5,   # batch size for evaluation
    warmup_steps=1000,               # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=200,               # How often to print logs
    do_train=True,                   # Perform training
    do_eval=True,                    # Perform evaluation
    evaluation_strategy="epoch",     # evalute after eachh epoch
    gradient_accumulation_steps=64,  # total number of steps before back propagation
    fp16=True,                       # Use mixed precision
    fp16_opt_level="02",             # mixed precision mode
    run_name="ProBert-BFD-MS",       # experiment name
    seed=3                           # Seed for experiment reproducibility 3x3
)

In [ ]:
from transformers import Trainer

#initialize Trainer
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tf_train_dataset,
    eval_dataset = tf_eval_dataset,
    compute_metrics = compute_metrics,
    tokenizer=tokenizer,
    callbacks=callbacks,
)

#train
trainer.train()

#save
trainer.save_model()

#eval
metrics = trainer.evaluate(eval_dataset=tf_eval_dataset)

AttributeError: ignored

In [ ]:
text = '''
주인공 강인구(하정우)는 '수리남에서 홍어가 많이 나는데 다 갖다버린다'는 친구 박응수(현봉식)의 얘기를 듣고 수리남산 홍어를 한국에 수출하기 위해 수리남으로 간다. 
국립수산과학원 측은 "실제로 남대서양에 홍어가 많이 살고 아르헨티나를 비롯한 남미 국가에서 홍어가 많이 잡힌다"며 "수리남 연안에도 홍어가 많이 서식할 것"이라고 설명했다.
그러나 관세청에 따르면 한국에 수리남산 홍어가 수입된 적은 없다. 
일각에선 "돈을 벌기 위해 수리남산 홍어를 구하러 간 설정은 개연성이 떨어진다"는 지적도 한다. 
드라마 배경이 된 2008~2010년에는 이미 국내에 아르헨티나, 칠레, 미국 등 아메리카산 홍어가 수입되고 있었기 때문이다. 
실제 조봉행 체포 작전에 협조했던 '협력자 K씨'도 홍어 사업이 아니라 수리남에 선박용 특수용접봉을 파는 사업을 하러 수리남에 갔었다.
'''

In [ ]:
text = '''
러시아가 2월24일 우크라이나를 침공해 12월26일로 306일째 침략 전쟁을 이어가고 있다. 이 전쟁으로 최대 10만명 이상의 목숨이 희생됐고 전세계는 핵전쟁도 배제할 수 없는 암울한 ‘신냉전 체제’로 빠져들었다. 전쟁은 장기화됐지만, 러시아와 우크라이나 어느 쪽도 물러날 기미를 보이지 않고 있어, 세계는 피로 물든 대결과 파괴로 2023년 새해를 맞게 됐다. 러시아의 우크라이나 침략은 유럽을 2차 세계대전 이후 최대·최악의 전쟁터로 바꿔놓으면서 크나큰 희생을 낳았다. 지금까지 민간인 사망자만 6702명(유엔 집계)에서 3만3430명(우크라이나 정부 추산)으로 추산된다. ‘부차 학살’ 등 러시아군의 잔혹한 전쟁 범죄도 잇따라 확인됐다. 우크라이나군 전사자는 1만명(우크라이나 정부 추산)에서 6만1207명(러시아 주장)으로 추정되고, 러시아군 전사자는 언론 등을 통해 공개된 자료 집계로만 1만229명이다. 미국 등이 추정하는 전체 군 사상자는 10만명 수준이다.전쟁이 길어질수록 어느 쪽도 압도적인 우세를 차지하지 못하는 ‘소모전’ 양상이 뚜렷해지고 있다. 러시아군은 침공 초기인 2월 말~3월 초 우크라이나 수도 키이우와 제2도시 하르키우 주변을 포위하면서 승리를 눈앞에 두는 듯했다. 하지만 우크라이나군의 강력한 반격에 직면해 3월 말~4월 초 키이우 주변에서 후퇴했고, 9월 초 하르키우주에서도 완전히 밀려났다. 11월11일에는 9개월 동안 점령했던 헤르손주의 주도 헤르손시도 다시 내줬다. 그 이후 전쟁은, 지상에서는 러시아군이 동부 도네츠크주에 모든 전력을 쏟아붓고, 공중에서는 미사일을 쏘아대며 전력망 등 우크라이나의 기반시설을 파괴하는 양상으로 전개되고 있다.러시아는 우크라이나 침공의 이유 중 하나로 북대서양조약기구(나토)의 확대에 따른 안보 위협을 내세웠지만, 결국 나토를 중심으로 한 서방의 결집이라는 역풍을 맞았다. 핀란드·스웨덴은 5월15일 74년 동안 고수하던 군사적 중립을 포기하고 나토 동시 가입을 선언했고, 회원국 정상들은 6월28일 두 나라의 가입을 승인했다. 미국의 21일 패트리엇 방공 미사일 등을 포함한 2조3800억원 규모 추가 군사 지원 발표에 이르기까지, 서방의 군사 지원도 이어지고 있다.러시아는 경제적으로도 전방위적인 압박을 받고 있다. 미국과 유럽연합(EU)은 기술 수출 금지, 중앙은행 자산 압류 등 제재 수위를 높여왔고, 지난 5일엔 러시아산 석유에 대한 ‘가격 상한제’를 실시해 러시아 경제의 핵심인 에너지 분야에 강력한 일격을 가했다.
'''

In [ ]:
model_inputs = tokenizer(
        text, max_length=max_input_length, truncation=True
    )

In [ ]:
predictions = model.generate([model_inputs['input_ids']], max_length = max_input_length)

In [ ]:
tokenizer.batch_decode(predictions, skip_special_tokens=True)

['러시아가 2월24일 우크라이나를 침공해 12월26일로 306일째 침략 전쟁을 이어가고 있다. 이 전쟁으로 최대 10만명 이상의 목숨이 희생됐고 전세계는 핵전쟁도 배제할 수 없는 암울한 신냉전 체제로 빠져들었다.']

In [28]:
import torch

!pip install keras-nlp --upgrade

!pip install rouge-score --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 337 kB 4.3 MB/s 
     |████████████████████████████████| 5.8 MB 77.9 MB/s 
     |████████████████████████        | 440.3 MB 120.9 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 588.3 MB 14 kB/s 
INFO: pip is looking at multiple versions of tensorflow-text to determine which version is compatible with other requirements. This could take a while.
     |████████████████████████████████| 5.9 MB 76.6 MB/s 
     |█████████████████▉              | 322.4 MB 143.3 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 578.1 MB 5.4 kB/s 
     |███████████████████▎            | 347.4 MB 73.5 MB/s eta 0:00:04

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 578.1 MB 4.4 kB/s 
     |████████████████████████████████| 4.6 MB 88.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=0560473b11a7c4bb910438d6b1640d4be18ef2e2e2324ee0863ebefb65ebf862
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge-score


In [43]:

# load weights into new model
model.load_weights("/content/drive/MyDrive/Alpaco_NLP/tf_model.h5")

In [52]:
text = '''
노형욱 장관, 철도역 방역ㆍ운행 안전에 만전강조\n 14일 서울역 KTX 철도역사열차 방역실태 전반에 걸쳐 현장 점검\n 철도특별사법경찰대에는 방역수칙 위반자에 대한 무관용 원칙 대응 주문 \n 노형욱 국토교통부 장관은 7월 14일 수도권 코로나19 방역의 관문이라고 할 수 있는 서울역을 방문하여, 철도역사 및 열차 방역실태 등을 점검하고, 관계자들을 격려하였다.\n 먼저, 노 장관은 한국철도공사로부터 수도권 거리두기 4단계 격상에 따른 전국 주요 역사 탑승 전 발열체크 등 강화된 철도분야 방역대책을 보고 받은 뒤, 코로나19 로 어려운 상황 속에서도 국민을 위해 힘써 주시는 노고에 감사드린다면서 관계자들을 격려하였다.\n 이어, KTX 방역현장을 둘러본 뒤, 지금까지 철도분야는 철두철미한 방역조치로 열차 내 감염사례가 단 한 건도 없었다라며, 강화된 방역대책이 승객에게 잘 안내되고, 이를 통해 실제 현장에서 잘 이행될 수 있도록 사명감을 갖고 총력을 기울여 줄 것을 강조하였다.\n 또한 방역으로 인해 자칫 운행 안전에 대한 경각심이 느슨해질 수 있다면서, 촘촘하고 치밀한 시설물 점검 및 차량의 정비를 통해 안전사고 예방에도 만전을 기해줄 것을 당부하였다.\n 이어, 서울지방철도특별사법경찰대를 방문한 노 장관은 철도시설 내 국민의 지킴이 역할과 해외입국자 KTX 전용칸 수송지원 업무를 차질 없이 수행하고 있는 철도경찰의 노고에 감사드린다면서,\n 코로나19로 인해 철도 승객 간 분쟁이 증가하는 등 철도경찰의 역할이 그 어느 때보다 중요한 때라며, 승객의 방역수칙 위반에 대해서는 무관용 원칙을 적용하여 국민들이 안심하고 철도를 이용할 수 있는 환경을 조성해 줄 것을 당부하였다
'''

max_input_length = 1024
max_target_length = 128

model_inputs = tokenizer(
        text, max_length=max_input_length, truncation=True
    )

predictions = model.generate(torch.tensor([model_inputs['input_ids']]), max_length = max_input_length)

pred = tokenizer.batch_decode(predictions, skip_special_tokens=True)
pred

['노 장관은 서울역을 방문하여 철도역사 및 열차 방역실태 등을 점검하고 관계자들을 격려하였다.']

In [ ]:
text = '''
노형욱 장관, 철도역 방역ㆍ운행 안전에 만전강조\n 14일 서울역 KTX 철도역사열차 방역실태 전반에 걸쳐 현장 점검\n 철도특별사법경찰대에는 방역수칙 위반자에 대한 무관용 원칙 대응 주문 \n 노형욱 국토교통부 장관은 7월 14일 수도권 코로나19 방역의 관문이라고 할 수 있는 서울역을 방문하여, 철도역사 및 열차 방역실태 등을 점검하고, 관계자들을 격려하였다.\n 먼저, 노 장관은 한국철도공사로부터 수도권 거리두기 4단계 격상에 따른 전국 주요 역사 탑승 전 발열체크 등 강화된 철도분야 방역대책을 보고 받은 뒤, 코로나19 로 어려운 상황 속에서도 국민을 위해 힘써 주시는 노고에 감사드린다면서 관계자들을 격려하였다.\n 이어, KTX 방역현장을 둘러본 뒤, 지금까지 철도분야는 철두철미한 방역조치로 열차 내 감염사례가 단 한 건도 없었다라며, 강화된 방역대책이 승객에게 잘 안내되고, 이를 통해 실제 현장에서 잘 이행될 수 있도록 사명감을 갖고 총력을 기울여 줄 것을 강조하였다.\n 또한 방역으로 인해 자칫 운행 안전에 대한 경각심이 느슨해질 수 있다면서, 촘촘하고 치밀한 시설물 점검 및 차량의 정비를 통해 안전사고 예방에도 만전을 기해줄 것을 당부하였다.\n 이어, 서울지방철도특별사법경찰대를 방문한 노 장관은 철도시설 내 국민의 지킴이 역할과 해외입국자 KTX 전용칸 수송지원 업무를 차질 없이 수행하고 있는 철도경찰의 노고에 감사드린다면서,\n 코로나19로 인해 철도 승객 간 분쟁이 증가하는 등 철도경찰의 역할이 그 어느 때보다 중요한 때라며, 승객의 방역수칙 위반에 대해서는 무관용 원칙을 적용하여 국민들이 안심하고 철도를 이용할 수 있는 환경을 조성해 줄 것을 당부하였다
'''

max_input_length = 1024
max_target_length = 128

model_inputs = tokenizer(
        text, max_length=max_input_length, truncation=True
    )

predictions = model.generate(torch.tensor([model_inputs['input_ids']]), max_length = max_input_length)

pred = tokenizer.batch_decode(predictions, skip_special_tokens=True)
pred

['주문 \nŕ형욱 국토교통부 장관은 7월 14일 수도권 코로나19 방역의 관문이라고 할 수 있는 서울역을 방문하여, 철도역사 및 열차 방역실태ㄛ 훔친']

In [ ]:
text = '''
노형욱 장관, 철도역 방역ㆍ운행 안전에 만전강조\n 14일 서울역 KTX 철도역사열차 방역실태 전반에 걸쳐 현장 점검\n 철도특별사법경찰대에는 방역수칙 위반자에 대한 무관용 원칙 대응 주문 \n 노형욱 국토교통부 장관은 7월 14일 수도권 코로나19 방역의 관문이라고 할 수 있는 서울역을 방문하여, 철도역사 및 열차 방역실태 등을 점검하고, 관계자들을 격려하였다.\n 먼저, 노 장관은 한국철도공사로부터 수도권 거리두기 4단계 격상에 따른 전국 주요 역사 탑승 전 발열체크 등 강화된 철도분야 방역대책을 보고 받은 뒤, 코로나19 로 어려운 상황 속에서도 국민을 위해 힘써 주시는 노고에 감사드린다면서 관계자들을 격려하였다.\n 이어, KTX 방역현장을 둘러본 뒤, 지금까지 철도분야는 철두철미한 방역조치로 열차 내 감염사례가 단 한 건도 없었다라며, 강화된 방역대책이 승객에게 잘 안내되고, 이를 통해 실제 현장에서 잘 이행될 수 있도록 사명감을 갖고 총력을 기울여 줄 것을 강조하였다.\n 또한 방역으로 인해 자칫 운행 안전에 대한 경각심이 느슨해질 수 있다면서, 촘촘하고 치밀한 시설물 점검 및 차량의 정비를 통해 안전사고 예방에도 만전을 기해줄 것을 당부하였다.\n 이어, 서울지방철도특별사법경찰대를 방문한 노 장관은 철도시설 내 국민의 지킴이 역할과 해외입국자 KTX 전용칸 수송지원 업무를 차질 없이 수행하고 있는 철도경찰의 노고에 감사드린다면서,\n 코로나19로 인해 철도 승객 간 분쟁이 증가하는 등 철도경찰의 역할이 그 어느 때보다 중요한 때라며, 승객의 방역수칙 위반에 대해서는 무관용 원칙을 적용하여 국민들이 안심하고 철도를 이용할 수 있는 환경을 조성해 줄 것을 당부하였다
'''

max_input_length = 1024
max_target_length = 128

model_inputs = tokenizer(
        text, max_length=max_input_length, truncation=True
    )

predictions = model2.model.generate(torch.tensor([model_inputs['input_ids']]), max_length = max_input_length)

pred = tokenizer.batch_decode(predictions, skip_special_tokens=True)
pred

['국토교통부 장관은 코로나19 방역수칙 위반에 대해 무관용 원칙을 적용하여 국민들이 안심하고 철도를 안전에 만전을 할 것을 당부하고 방역수칙 위반에 대해서는 무관용 원칙을 적용하여 안전에 만전을 강강을조해줄 것을 당부했다.']

In [53]:

from tensorflow import keras
import rouge_score
import keras_nlp
from keras_nlp.metrics.rouge_base import RougeBase
rouge_l = keras_nlp.metrics.RougeL()
y_true = "노형욱 국토교통부 장관은 7월 14일 수도권 코로나19 방역의 관문이라고 할 수 있는 서울역을 방문하여, 철도역사 및 열차 방역실태 등을 점검하고, 관계자들을 격려하였다. 또한 방역으로 인해 자칫 운행 안전에 대한 경각심이 느슨해질 수 있다면서, 촘촘하고 치밀한 시설물 점검 및 차량의 정비를 통해 안전사고 예방에도 만전을 기해줄 것을 당부하였다."
y_pred = '노 장관은 서울역을 방문하여 철도역사 및 열차 방역실태 등을 점검하고 관계자들을 격려하였다.'
rouge_l(y_true, y_pred)

{'precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>,
 'recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>,
 'f1_score': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>}

In [54]:
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu


def bleu(ref, gen):
    ''' 
    calculate pair wise bleu score. uses nltk implementation
    Args:
        references : a list of reference sentences 
        candidates : a list of candidate(generated) sentences
    Returns:
        bleu score(float)
    '''
    ref_bleu = []
    gen_bleu = []
    for l in gen:
        gen_bleu.append(l.split())
    for i,l in enumerate(ref):
        ref_bleu.append([l.split()])
    cc = SmoothingFunction()
    score_bleu = corpus_bleu(ref_bleu, gen_bleu, weights=(0, 1, 0, 0), smoothing_function=cc.method4)
    return score_bleu

y_pred = '노 장관은 서울역을 방문하여 철도역사 및 열차 방역실태 등을 점검하고 관계자들을 격려하였다.'
bleu([y_true], [y_pred])

0.024597166464918913